# Projet Deep Reinforcement Learning

##  l'Environnement grid World

#### Import des biblio

In [17]:
import numpy as np
import time
from typing import Callable

#### Initiale les variables d'environnement

In [18]:
width = 4
height = 4
num_states = width * height
S = np.arange(num_states)
A = np.arange(4)  # 0: left, 1: Right, 2: Up, 3: Down
T = np.array([width - 1, num_states - 1])
P = np.zeros((len(S), len(A), len(S), 2))

In [19]:
for s in S:
    if (s % width) == 0:
        P[s, 0, s, 0] = 1.0
    else:
        P[s, 0, s - 1, 0] = 1.0
    if (s + 1) % width == 0:
        P[s, 1, s, 0] = 1.0
    else:
        P[s, 1, s + 1, 0] = 1.0
    if s < width:
        P[s, 2, s, 0] = 1.0
    else:
        P[s, 2, s - width, 0] = 1.0
    if s >= (num_states - width):
        P[s, 3, s, 0] = 1.0
    else:
        P[s, 3, s + width, 0] = 1.0

P[width - 1, :, :, 0] = 0.0
P[num_states - 1, :, :, 0] = 0.0

P[:, :, width - 1, 1] = -5.0
P[:, :, num_states - 1, 1] = 1.0

#### definir les fonction d'environnement

In [20]:
def reset() -> int:
    return 0


def is_terminal(state: int) -> bool:
    return state in T


def step(state: int, action: int) -> (int, float, bool):
    assert not is_terminal(state)
    next_state = np.random.choice(S, p=P[state, action, :, 0])
    reward = P[state, action, next_state, 1]
    return next_state, reward

#### definire le policy randon uniform

In [21]:
def tabular_random_uniform_policy(state_size: int, action_size: int) -> np.ndarray:
    assert action_size > 0
    return np.ones((state_size, action_size,)) / action_size

#### Implement l'algorithms iterative policy evaluation

In [22]:
def iterative_policy_evaluation(
        S: np.ndarray,
        A: np.ndarray,
        P: np.ndarray,
        T: np.ndarray,
        Pi: np.ndarray,
        gamma: float = 0.99,
        theta: float = 0.00001
) -> np.ndarray:
    assert theta > 0
    assert 0 <= gamma <= 1
    V = np.random.random((S.shape[0],))
    V[T] = 0.0
    while True:
        delta = 0
        for s in S:
            v_temp = V[s]
            new_v = 0
            for a in A:
                for s_p in S:
                    new_v += Pi[s, a] * P[s, a, s_p, 0] * (
                            P[s, a, s_p, 1] + gamma * V[s_p]
                    )
            V[s] = new_v
            delta = np.maximum(delta, np.abs(v_temp - new_v))
        if delta < theta:
            break
    return V

#### test l'algorithms iterative policy evaluation

In [23]:
print("Evaluation policy random :")
Pi = tabular_random_uniform_policy(S.shape[0], A.shape[0])
V = iterative_policy_evaluation(S, A, P, T, Pi)
print(V)

Evaluation policy random :
[-1.95551426 -2.28653885 -3.1369096   0.         -1.70353726 -1.85959763
 -2.20043799 -2.69685022 -1.36435555 -1.32302373 -1.19730831 -0.94857625
 -1.1216561  -0.92430044 -0.36557768  0.        ]


#### Implement l'algorithms policy iteration

In [24]:
def policy_iteration(
        S: np.ndarray,
        A: np.ndarray,
        P: np.ndarray,
        T: np.ndarray,
        gamma: float = 0.99,
        theta: float = 0.00001
) -> (np.ndarray, np.ndarray):
    Pi = tabular_random_uniform_policy(S.shape[0], A.shape[0])
    while True:
        V = iterative_policy_evaluation(S, A, P, T, Pi, gamma, theta)
        policy_stable = True
        for s in S:
            old_action = np.argmax(Pi[s])
            best_action = 0
            best_action_score = -9999999999
            for a in A:
                tmp_sum = 0
                for s_p in S:
                    tmp_sum += Pi[s, a] * P[s, a, s_p, 0] * (
                            P[s, a, s_p, 1] + gamma * V[s_p]
                    )
                if tmp_sum > best_action_score:
                    best_action = a
                    best_action_score = tmp_sum
            Pi[s] = 0.0
            Pi[s, best_action] = 1.0
            if old_action != best_action:
                policy_stable = False
        if policy_stable:
            break
    V = iterative_policy_evaluation(S, A, P, T, Pi, gamma, theta)
    return V, Pi

#### test l'algorithms policy iteration

In [25]:
t1 = time.time()
V, Pi = policy_iteration(S, A, P, T)
print(V)
print(Pi)
print(f"time : {time.time() - t1}")

[0.95099005 0.96059601 0.970299   0.         0.96059601 0.970299
 0.9801     0.99       0.970299   0.9801     0.99       1.
 0.9801     0.99       1.         0.        ]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
time : 0.16193222999572754


#### Implement l'algorithms value iteration

In [26]:
def value_iteration(
    S: np.ndarray,
    A: np.ndarray,
    P: np.ndarray,
    T: np.ndarray,
    gamma: float = 0.99,
    theta: float = 0.00001
) -> np.ndarray:
    V = np.random.random((S.shape[0],))
    V[T] = 0.0
    while True:
        delta = 0
        for s in S:
            v_temp = V[s]
            v_max = -99999999
            for a in A:
                v_max_temp = 0
                for s_p in S:
                    v_max_temp += P[s, a, s_p, 0] * (
                            P[s, a, s_p, 1] + gamma * V[s_p]
                    )
                if(v_max < v_max_temp):
                    v_max = v_max_temp
            V[s] = v_max
            delta = np.maximum(delta, np.abs(v_temp - v_max))

        if delta < theta:
            break
    
    Pi = tabular_random_uniform_policy(S.shape[0], A.shape[0])
    for s in S:
        old_action = np.argmax(Pi[s])
        best_action = 0
        best_action_score = -9999999999
        for a in A:
            tmp_sum = 0
            for s_p in S:
                tmp_sum += Pi[s, a] * P[s, a, s_p, 0] * (
                        P[s, a, s_p, 1] + gamma * V[s_p]
                )
            if tmp_sum > best_action_score:
                best_action = a
                best_action_score = tmp_sum
        Pi[s] = 0.0
        Pi[s, best_action] = 1.0
    return V, Pi

#### test l'algorithms value iteration

In [27]:
t1 = time.time()
V, Pi = value_iteration(S, A, P, T)
print(V)
print(Pi)
print(f"le time d'execution : {time.time() - t1}")

[0.95099005 0.96059601 0.970299   0.         0.96059601 0.970299
 0.9801     0.99       0.970299   0.9801     0.99       1.
 0.9801     0.99       1.         0.        ]
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
le time d'execution : 0.011749744415283203


#### definir un utile pour simules des step

In [28]:
def step_until_the_end_of_the_episode_and_generate_trajectory(
        s0: int,
        pi: np.ndarray,
        step_func: Callable,
        is_terminal_func: Callable,
        max_steps: int = 10
) -> ([int], [int], [int], [float]):
    s_list = []
    a_list = []
    s_p_list = []
    r_list = []
    st = s0
    actions = np.arange(pi.shape[1])
    step = 0
    while not is_terminal_func(st) and step < max_steps:
        at = np.random.choice(actions, p=pi[st])
        st_p, rt_p = step_func(st, at)
        s_list.append(st)
        a_list.append(at)
        s_p_list.append(st_p)
        r_list.append(rt_p)
        st = st_p
        step += 1

    return s_list, a_list, s_p_list, r_list

#### Implement l'algorithms monte carlo es

In [29]:
def monte_carlo_es(
        states_count: int,
        actions_count: int,
        step_func: Callable,
        is_terminal_func: Callable,
        max_episodes: int = 1000,
        max_steps_per_episode: int = 10,
        gamma: float = 0.99
) -> (np.ndarray, np.ndarray):
    pi = tabular_random_uniform_policy(states_count, actions_count)
    states = np.arange(states_count)
    actions = np.arange(actions_count)

    Q = np.random.random((states_count, actions_count))

    for s in states:
        if is_terminal_func(s):
            Q[s, :] = 0.0
            pi[s, :] = 0.0

    returns = np.zeros((states_count, actions_count))
    returns_count = np.zeros((states_count, actions_count))

    for episode_id in range(max_episodes):
        s0 = np.random.choice(states)

        if is_terminal_func(s0):
            episode_id -= 1
            continue

        a0 = np.random.choice(actions)

        s1, r1 = step_func(s0, a0)

        s_list, a_list, _, r_list = step_until_the_end_of_the_episode_and_generate_trajectory(s1, pi, step_func,
                                                                                              is_terminal_func,
                                                                                              max_steps_per_episode)
        s_list.insert(0, s0)
        a_list.insert(0, a0)
        r_list.insert(0, r1)

        G = 0.0
        for t in reversed(range(len(s_list))):
            G = gamma * G + r_list[t]
            st = s_list[t]
            at = a_list[t]
            if (st, at) in zip(s_list[0:t], a_list[0:t]):
                continue

            returns[st, at] += G
            returns_count[st, at] += 1
            Q[st, at] = returns[st, at] / returns_count[st, at]
            pi[st, :] = 0.0
            pi[st, np.argmax(Q[st, :])] = 1.0
    return Q, pi

#### test l'algorithms monte_carlo_es

In [30]:
t1 = time.time()
Q, Pi = monte_carlo_es(len(S), len(A), step, is_terminal,
                        max_episodes=10000, max_steps_per_episode=100)
print(Q)
print(Pi)
print(f"le time d'execution : {time.time() - t1}")

[[ 0.92191482  0.94804383  0.92426197  0.93599526]
 [ 0.93145844  0.95168495  0.94316664  0.9602388 ]
 [ 0.94764964 -5.          0.95507534  0.96953826]
 [ 0.          0.          0.          0.        ]
 [ 0.94171607  0.95866071  0.93186564  0.94574414]
 [ 0.9423193   0.97010728  0.94262422  0.96695723]
 [ 0.95761182  0.97695486  0.95715985  0.9801    ]
 [ 0.96706397  0.9801     -5.          0.99      ]
 [ 0.95340921  0.970299    0.94129721  0.96307044]
 [ 0.96016024  0.97596308  0.9588069   0.9801    ]
 [ 0.97017522  0.99        0.96679044  0.98682418]
 [ 0.9801      0.99        0.9801      1.        ]
 [ 0.96369833  0.97997554  0.95938558  0.96882653]
 [ 0.970299    0.99        0.96984192  0.9801    ]
 [ 0.9801      1.          0.97672231  0.98603322]
 [ 0.          0.          0.          0.        ]]
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0

#### Implement l'algorithms on policy first visit monte carlo

In [31]:
def on_policy_first_visit_monte_carlo(
        states_count: int,
        actions_count: int,
        reset_func: Callable,
        step_func: Callable,
        is_terminal_func: Callable,
        max_episodes: int = 1000,
        max_steps_per_episode: int = 10,
        gamma: float = 0.99,
        epsilon: float = 0.1
) -> (np.ndarray, np.ndarray):
    pi = tabular_random_uniform_policy(states_count, actions_count)
    states = np.arange(states_count)
    actions = np.arange(actions_count)

    Q = np.random.random((states_count, actions_count))

    for s in states:
        if is_terminal_func(s):
            Q[s, :] = 0.0
            pi[s, :] = 0.0

    returns = np.zeros((states_count, actions_count))
    returns_count = np.zeros((states_count, actions_count))

    for episode_id in range(max_episodes):
        s0 = reset_func()

        s_list, a_list, _, r_list = step_until_the_end_of_the_episode_and_generate_trajectory(s0, pi, step_func,
                                                                                              is_terminal_func,
                                                                                              max_steps_per_episode)

        G = 0.0
        for t in reversed(range(len(s_list))):
            G = gamma * G + r_list[t]
            st = s_list[t]
            at = a_list[t]
            if (st, at) in zip(s_list[0:t], a_list[0:t]):
                continue

            returns[st, at] += G
            returns_count[st, at] += 1
            Q[st, at] = returns[st, at] / returns_count[st, at]
            pi[st, :] = epsilon / actions_count
            pi[st, np.argmax(Q[st, :])] = 1.0 - epsilon + epsilon / actions_count
    return Q, pi

#### test l'algorithms on policy first visit monte carlo

In [32]:
t1 = time.time()
Q, Pi = on_policy_first_visit_monte_carlo(len(S), len(A),
                                            reset,
                                            step,
                                            is_terminal,
                                            max_episodes=10000, max_steps_per_episode=100)
print(Q)
print(Pi)
print(f"le time d'execution : {time.time() - t1}")

[[ 0.9027372   0.87052012  0.90551516  0.94376221]
 [ 0.91864514  0.31641809 -0.30543843 -0.64418598]
 [-1.60866294 -5.          0.9060982   0.96664098]
 [ 0.          0.          0.          0.        ]
 [ 0.91384971  0.93640496  0.90541895  0.95480436]
 [ 0.23784166  0.94798949  0.5652377  -0.19208039]
 [ 0.95111707  0.66644711  0.22177151  0.97784569]
 [ 0.970299    0.51618868 -5.          0.98858925]
 [ 0.95420505  0.93965067  0.94322714  0.96595533]
 [ 0.93533893  0.61440631  0.67262906  0.97762651]
 [ 0.95869034  0.98904664  0.65978263  0.98548377]
 [ 0.97745364  0.99        0.9771695   1.        ]
 [ 0.96454444  0.97702392  0.95400809  0.96478263]
 [ 0.96524858  0.98838785  0.96334362  0.97639912]
 [ 0.97707622  1.          0.97761554  0.98813239]
 [ 0.          0.          0.          0.        ]]
[[0.025 0.025 0.025 0.925]
 [0.925 0.025 0.025 0.025]
 [0.025 0.025 0.025 0.925]
 [0.    0.    0.    0.   ]
 [0.025 0.025 0.025 0.925]
 [0.025 0.925 0.025 0.025]
 [0.025 0.025 0.025 0

#### Implement l'algorithms off policy monte carlo control

In [33]:
def off_policy_monte_carlo_control(
        states_count: int,
        actions_count: int,
        reset_func: Callable,
        step_func: Callable,
        is_terminal_func: Callable,
        max_episodes: int = 1000,
        max_steps_per_episode: int = 10,
        gamma: float = 0.99,
        epsilon: float = 0.1,
        epsilon_greedy_behaviour_policy: bool = False
) -> (np.ndarray, np.ndarray):
    b = tabular_random_uniform_policy(states_count, actions_count)
    pi = tabular_random_uniform_policy(states_count, actions_count)
    states = np.arange(states_count)

    Q = np.random.random((states_count, actions_count))

    for s in states:
        if is_terminal_func(s):
            Q[s, :] = 0.0
            pi[s, :] = 0.0
        pi[s, :] = 0
        pi[s, np.argmax(Q[s, :])] = 1.0

    C = np.zeros((states_count, actions_count))

    for episode_id in range(max_episodes):
        if epsilon_greedy_behaviour_policy:
            for s in states:
                b[s, :] = epsilon / actions_count
                b[s, np.argmax(Q[s, :])] = 1.0 - epsilon + epsilon / actions_count

        s0 = reset_func()

        s_list, a_list, _, r_list = step_until_the_end_of_the_episode_and_generate_trajectory(s0, b, step_func,
                                                                                              is_terminal_func,
                                                                                              max_steps_per_episode)

        G = 0.0
        W = 1
        for t in reversed(range(len(s_list))):
            G = gamma * G + r_list[t]
            st = s_list[t]
            at = a_list[t]
            C[st, at] += W

            Q[st, at] += W / C[st, at] * (G - Q[st, at])
            pi[st, :] = 0
            pi[st, np.argmax(Q[st, :])] = 1.0
            if np.argmax(Q[st, :]) != at:
                break
            W = W / b[st, at]
    return Q, pi

#### test l'algorithms off policy monte carlo control

In [34]:
t1 = time.time()
Q, Pi = off_policy_monte_carlo_control(len(S), len(A),
                                           reset,
                                           step,
                                           is_terminal,
                                           max_episodes=10000, max_steps_per_episode=100)
print(Q)
print(Pi)
print(f"le time d'execution : {time.time() - t1}")

[[ 0.93918721  0.95045966  0.94033228  0.94950645]
 [ 0.94102067  0.95974396  0.95075793  0.96048232]
 [ 0.95099005 -5.          0.96003349  0.970299  ]
 [ 0.          0.          0.          0.        ]
 [ 0.95012847  0.9598703   0.94094413  0.95883345]
 [ 0.94895129  0.96931125  0.95006225  0.96924241]
 [ 0.95611422  0.97840139  0.95834988  0.9801    ]
 [ 0.96885725  0.9801     -5.          0.98886598]
 [ 0.9589312   0.96906654  0.94705938  0.96769766]
 [ 0.95802319  0.9798931   0.95827874  0.9786632 ]
 [ 0.96883403  0.99        0.96931125  0.98970009]
 [ 0.97949388  0.9875495   0.9794044   1.        ]
 [ 0.96849748  0.97766334  0.95706517  0.96925454]
 [ 0.96315695  0.98884265  0.96887974  0.97795536]
 [ 0.97702759  1.          0.9801      0.99      ]
 [ 0.          0.          0.          0.        ]]
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0